In [1]:
!pip install --upgrade --quiet  langchain langchain-experimental langchain-openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai-whisper 20230314 requires tiktoken==0.3.1, but you have tiktoken 0.5.2 which is incompatible.
grobid-client 0.8.3 requires httpx<0.23.0,>=0.15.4, but you have httpx 0.26.0 which is incompatible.
crewai 0.1.24 requires langchain<0.0.355,>=0.0.354, but you have langchain 0.1.0 which is incompatible.


In [2]:
import dotenv
dotenv.load_dotenv()

True

In [4]:
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 9.0 MB/s eta 0:00:00 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.9-cp311-cp311-macosx_11_0_arm64.whl size=132042 sha256=ff696c30f747e588d1d8c7f973806804d746dd17c687011c06ab36052807ec93
  Stored in directory: /Users/heewungsong/Library/Caches/pip/wheels/ab/34/b9/78ebef1b3220b4840ee482461e738566c3c9165d2b5c914f51
Successfully built psycopg2


In [1]:
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import OpenAI
import os

db = SQLDatabase.from_uri(os.environ['DB_URL'])
llm = OpenAI(model="gpt-3.5-turbo-1106", temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [2]:
db_chain.run("How many users are there?")

/Users/heewungsong/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many users are there?
SQLQuery:

NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

In [12]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

In [13]:
chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "How many users are there"})
print(response)

SELECT COUNT(*) FROM users


In [14]:
db.run(response)

'[(16,)]'

In [15]:
chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "How many under 30 age users are there"})
print(response)

SELECT COUNT(*) 
FROM users 
WHERE age < 30


In [16]:
db.run(response)

'[(2,)]'

In [17]:
chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "국적이 대한민국인 유저는 누가있어?"})
print(response)

SELECT name FROM users WHERE nation = 'Korea';


In [25]:
from langchain.agents import create_sql_agent

# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain_community.agent_toolkits import SQLDatabaseToolkit

llm = ChatOpenAI(model_name="gpt-4", temperature=0)
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [20]:
# agent_executor.run("유저중에 20대 미만인 사람은 몇명있어?")
agent_executor.run("How many under 30 age users are there?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input:

Observation: chat_messages, gpt_message, link_box_messages, link_boxs, messages, next_users, users
Thought: I should query the schema of the users table to see what columns are available.
Action: sql_db_schema
Action Input: users
Observation: 
CREATE TABLE next_auth.users (
	id UUID DEFAULT uuid_generate_v4() NOT NULL, 
	name TEXT, 
	email TEXT, 
	"emailVerified" TIMESTAMP WITH TIME ZONE, 
	image TEXT, 
	CONSTRAINT users_pkey PRIMARY KEY (id), 
	CONSTRAINT email_unique UNIQUE (email)
)

/*
3 rows from users table:
id	name	email	emailVerified	image
17fc53b0-fddb-47be-8968-9b3e7f6ca259	test1	tes1@example.com	2024-01-02 01:06:22+00:00	None
81c76d8a-d384-42d1-b1f7-26a9611e75ca	kofsitho	kofsitho@gmail.com	None	https://avatars.githubusercontent.com/u/11041161?v=4
*/


CREATE TABLE users (
	id BIGINT GENERATED BY DEFAULT AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 9223372036854775807 CACHE 1 NO 

'2'

In [21]:
agent_executor.run("Are there any users under 20s?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input:

Observation: chat_messages, gpt_message, link_box_messages, link_boxs, messages, next_users, users
Thought: I should query the schema of the users table to see what columns I can use to filter by age.
Action: sql_db_schema
Action Input: users
Observation: 
CREATE TABLE next_auth.users (
	id UUID DEFAULT uuid_generate_v4() NOT NULL, 
	name TEXT, 
	email TEXT, 
	"emailVerified" TIMESTAMP WITH TIME ZONE, 
	image TEXT, 
	CONSTRAINT users_pkey PRIMARY KEY (id), 
	CONSTRAINT email_unique UNIQUE (email)
)

/*
3 rows from users table:
id	name	email	emailVerified	image
17fc53b0-fddb-47be-8968-9b3e7f6ca259	test1	tes1@example.com	2024-01-02 01:06:22+00:00	None
81c76d8a-d384-42d1-b1f7-26a9611e75ca	kofsitho	kofsitho@gmail.com	None	https://avatars.githubusercontent.com/u/11041161?v=4
*/


CREATE TABLE users (
	id BIGINT GENERATED BY DEFAULT AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 922337203685477580

"I don't know"

In [31]:
agent_executor.run("find the row that the title is '네이버 페이 증권' in the link_box table. show me the whole row data")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: chat_messages, gpt_message, link_box_messages, link_boxs, messages, next_users, users
Thought:The 'link_boxs' table seems to be the most relevant to the question. I should check its schema to understand its structure and the fields it contains.
Action: sql_db_schema
Action Input: "link_boxs"
Observation: 
CREATE TABLE link_boxs (
	id BIGINT GENERATED BY DEFAULT AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 9223372036854775807 CACHE 1 NO CYCLE), 
	"user" UUID NOT NULL, 
	url TEXT NOT NULL, 
	title TEXT NOT NULL, 
	description TEXT, 
	image TEXT, 
	content TEXT NOT NULL, 
	created_at TIMESTAMP WITH TIME ZONE DEFAULT now() NOT NULL, 
	host_url TEXT NOT NULL, 
	keywords VARCHAR[], 
	summary TEXT, 
	category TEXT, 
	etc JSONB, 
	questions TEXT[], 
	CONSTRAINT link_boxs_pkey PRIMARY KEY (id), 
	CONSTRAINT link_boxs_user_fkey FOREIGN KEY("user") REFERENCES next_users (id)
)

/*
3 ro

"I'm sorry, but there is no row in the 'link_boxs' table where the title is '네이버 페이 증권'."

In [23]:
agent_executor.run("Describe the link_boxs table")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input:

Observation: chat_messages, gpt_message, link_box_messages, link_boxs, messages, next_users, users
Thought: I should query the schema of the link_boxs table.
Action: sql_db_schema
Action Input: link_boxs
Observation: 
CREATE TABLE link_boxs (
	id BIGINT GENERATED BY DEFAULT AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 9223372036854775807 CACHE 1 NO CYCLE), 
	"user" UUID NOT NULL, 
	url TEXT NOT NULL, 
	title TEXT NOT NULL, 
	description TEXT, 
	image TEXT, 
	content TEXT NOT NULL, 
	created_at TIMESTAMP WITH TIME ZONE DEFAULT now() NOT NULL, 
	host_url TEXT NOT NULL, 
	keywords VARCHAR[], 
	summary TEXT, 
	category TEXT, 
	etc JSONB, 
	questions TEXT[], 
	CONSTRAINT link_boxs_pkey PRIMARY KEY (id), 
	CONSTRAINT link_boxs_user_fkey FOREIGN KEY("user") REFERENCES next_users (id)
)

/*
3 rows from link_boxs table:
id	user	url	title	description	image	content	created_at	host_url	keywords	summar

'CREATE TABLE link_boxs (\n\tid BIGINT GENERATED BY DEFAULT AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 9223372036854775807 CACHE 1 NO CYCLE), \n\t"user" UUID NOT NULL, \n\turl TEXT NOT NULL, \n\ttitle TEXT NOT NULL, \n\tdescription TEXT, \n\timage TEXT, \n\tcontent TEXT NOT NULL, \n\tcreated_at TIMESTAMP WITH TIME ZONE DEFAULT now() NOT NULL, \n\thost_url TEXT NOT NULL, \n\tkeywords VARCHAR[], \n\tsummary TEXT, \n\tcategory TEXT, \n\tetc JSONB, \n\tquestions TEXT[], \n\tCONSTRAINT link_boxs_pkey PRIMARY KEY (id), \n\tCONSTRAINT link_boxs_user_fkey FOREIGN KEY("user") REFERENCES next_users (id)\n)\n\n/*\n3 rows from link_boxs table:\nid\tuser\turl\ttitle\tdescription\timage\tcontent\tcreated_at\thost_url\tkeywords\tsummary\tcategory\tetc\tquestions\n14\t81c76d8a-d384-42d1-b1f7-26a9611e75ca\thttps://www.bloter.net\t삼성전자, 반도체 적자폭은 축'

In [32]:
agent_executor.run("In the link_box table, find the data related to the year-end settlement and let me know what the year-end settlement is")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: chat_messages, gpt_message, link_box_messages, link_boxs, messages, next_users, users
Thought:The link_boxs table seems to be the most relevant to the question. I should check its schema to understand its structure and the type of data it contains.
Action: sql_db_schema
Action Input: "link_boxs"
Observation: 
CREATE TABLE link_boxs (
	id BIGINT GENERATED BY DEFAULT AS IDENTITY (INCREMENT BY 1 START WITH 1 MINVALUE 1 MAXVALUE 9223372036854775807 CACHE 1 NO CYCLE), 
	"user" UUID NOT NULL, 
	url TEXT NOT NULL, 
	title TEXT NOT NULL, 
	description TEXT, 
	image TEXT, 
	content TEXT NOT NULL, 
	created_at TIMESTAMP WITH TIME ZONE DEFAULT now() NOT NULL, 
	host_url TEXT NOT NULL, 
	keywords VARCHAR[], 
	summary TEXT, 
	category TEXT, 
	etc JSONB, 
	questions TEXT[], 
	CONSTRAINT link_boxs_pkey PRIMARY KEY (id), 
	CONSTRAINT link_boxs_user_fkey FOREIGN KEY("user") REFERENCES next_users (id)
)

/*


"There is no data related to the 'year-end settlement' in the link_boxs table."